Monte Carlo simulation for determining the RMSE and R-Squared

In [3]:
from joblib import Parallel, delayed
import multiprocessing
from scipy.optimize import OptimizeWarning
import warnings
import pandas as pd
import scipy.optimize as sc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import logging
np.seterr(divide='ignore')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Function to handle numpy errors and log them
def handle_numpy_errors(err, flag):
    logging.error(f"Numpy error: {err}, Flag: {flag}")

# Configure numpy to use the error handler
np.seterrcall(handle_numpy_errors)
np.seterr(all='call')

# Set up logging to file
logging.basicConfig(filename='error_log.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to log warnings
def log_warning(message, category, filename, lineno, file=None, line=None):
    logging.error(f'Warning: {message}, Category: {category.__name__}, Filename: {filename}, Line: {lineno}')

# This function should be called at the beginning of the parallelized function
def setup_warning_logging():
    warnings.showwarning = log_warning
    warnings.simplefilter('always', RuntimeWarning)  # Adjust as needed

# Load data from CSV file
file2 = pd.read_csv("./../../Output/LowPPMMatrix.csv")

file2=file2[file2['Target PPM']!=150]
file2=file2[file2['SensorID']==0]
#calculate abso.lute humidity in PPM
xDataRH = file2.loc[:, 'RelativeHumidity']
xDataTemp = file2.loc[:, 'Temperature']
P_actual_hPa = .8 * 1013.25
e_sat_standard = 6.112 * np.exp((17.67 * xDataTemp) / (xDataTemp + 243.5))
e_sat_actual = e_sat_standard * (P_actual_hPa / 1013.25)
absoluteHumidity = 1000*((xDataRH/100)*e_sat_actual)/(461.5*(xDataTemp+ 273.15))
file2['AbsoluteHumidity'] = absoluteHumidity
# Separate the independent and dependent variables
X = file2[['Resistance', 'RelativeHumidity', 'Temperature']]
y = file2['Target PPM'] + 4.21

p0 = [1] * 10
p1 = [1] * 4

def funkEQ(X, a,b,c,d,e,f,g,h,i,j):
    R, H, T = X
    with np.errstate(over='ignore'):
        stuff= a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1*T*f)/(H**g))*h)+i)+j
    return stuff

def bastEQ(X, a,b,c,d):
    R, H, T = X
    with np.errstate(over='ignore'):
        stuff= a*R**b+c*H*(a*R**b)+d
    return stuff

def residual(params, X, y):
    return np.sum((y - funkEQ(X, *params))**2)


# Function to calculate RMSE and R-squared
def calculate_performance_metrics(y_test, y_pred):
    rmse_value = mean_squared_error(y_test, y_pred, squared=False)
    ss_res = np.sum((y_test - y_pred) ** 2)
    ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    return rmse_value, r_squared

# Generalized model fitting function
def fit_model_general(iteration, X, y, params, equation):
    setup_warning_logging()
    warnings.filterwarnings("ignore", category=OptimizeWarning)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=iteration)
    popt, _ = sc.curve_fit(equation, (X_train['Resistance'], X_train[humidity_type], X_train['Temperature']), y_train, params, maxfev=1000000)
    y_pred = equation((X_test['Resistance'], X_test[humidity_type], X_test['Temperature']), *popt)
    return calculate_performance_metrics(y_test, y_pred)

# Run model fitting for both types of humidity
humidity_types = ['RelativeHumidity', 'AbsoluteHumidity']
equations = [(funkEQ, p0), (bastEQ, p1)]
num_cores = multiprocessing.cpu_count()
i = 1000
#tqdm()
for humidity_type in humidity_types:
    X = file2[['Resistance', humidity_type, 'Temperature']]
    for eq_name, params in equations:
        eq_name_str = 'Funk' if eq_name == funkEQ else 'Bastviken'
        results = Parallel(n_jobs=num_cores)(delayed(fit_model_general)(j, X, y, params, eq_name) for j in tqdm(range(i), mininterval=1, ncols=100, leave=False, bar_format='{l_bar}{bar}{r_bar}'))
        rmse_values, r_squared_values = zip(*results)
        average_rmse = np.mean(rmse_values)
        average_rsqrd = np.mean(r_squared_values)
        print(f'{eq_name_str} Equation {humidity_type} Average RMSE over {i} iterations: {average_rmse}')
        print(f'{eq_name_str} Equation {humidity_type} Average R-Squared over {i} iterations: {average_rsqrd}')

  4%|██▏                                                          | 36/1000 [00:01<00:28, 33.88it/s]ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/356620135

Funk Equation RelativeHumidity Average RMSE over 1000 iterations: 7.816161939718231
Funk Equation RelativeHumidity Average R-Squared over 1000 iterations: 0.6834920742264811


Bastviken Equation RelativeHumidity Average RMSE over 1000 iterations: 12.528436103195126
Bastviken Equation RelativeHumidity Average R-Squared over 1000 iterations: 0.25496346216203386


 74%|███████████████████████████████████████████▉               | 744/1000 [00:05<00:01, 134.38it/s]ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/3566201350.py, Line: 59
ERROR:root:Warning: divide by zero encountered in divide, Category: RuntimeWarning, Filename: /var/folders/1g/qs1wt1d52t1fp_9wj2_z6vlr0000gn/T/ipykernel_52032/356620135

Funk Equation AbsoluteHumidity Average RMSE over 1000 iterations: 8.624454351575796
Funk Equation AbsoluteHumidity Average R-Squared over 1000 iterations: 0.5080633938162917


Bastviken Equation AbsoluteHumidity Average RMSE over 1000 iterations: 15.056413187927944
Bastviken Equation AbsoluteHumidity Average R-Squared over 1000 iterations: -0.08317733738795637


New Funk Equations Data (1000)



 Funk Equation Iterations
a*R+b
a*np.exp(-1*R*b+c)+d
a*R**b+c
a*R**b+c*H*(a*R**b+c)+d  (Basically Bastviken)
(a*np.exp(-1*R*b+c)+d)+f*H*(a*np.exp(-1*R*b+c)+d)+g
a*np.exp(-1*R*b+c)+d*np.exp(-1*H*f+g)+h (Funk Equation)
a*np.exp((-1*R*b+c)+(-1*H*d+e))+f 4.2
                                                          W/1000  W/500   UV500   UV1000
a**((-1*R*b+c)+(-1*H*d+e))+f 6.4                          87.79   42.64   62.74   116.35
a**((-1*R*b)+(-1*H*c)+d)+e   6.5                          87.79

a**((((-1*R)/(H**b))*c)+(-1*H*d)+e)+f 7.1                 74.09   35.76   62.37   113.46


a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+f)+g 8.1
a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1)/(T*f*H**g))*h)+i)+j 8.2
a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1*T*f)/(H**g))*h)+i)+j 8.3                           5.764

a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1*T*f)/(H**g))*h)+i)+j*np.exp(-1*T*k)+l 9.1


0.97**((((-1*R)/(H**(-0.66)))*c)+(-1*H)+(1.21*T)+(((-1.22*T)/(H**0.23))*1.25)+-178.26)+j            5.865


6.33 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o

6.18 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o*np.log(H) +q

5.75 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o*(R*T) +q

